In [308]:

with open("../data/the_verdict.txt", "r") as f:
    raw_text = f.read()

print(f"Total characters: {len(raw_text)}")
print(f"Total words: {len(raw_text.split())}")

Total characters: 20479
Total words: 3634


In [309]:
import re

text = "Hello world this is some sample text."
result = re.split(r"(\s)", text)
print(result)

['Hello', ' ', 'world', ' ', 'this', ' ', 'is', ' ', 'some', ' ', 'sample', ' ', 'text.']


In [310]:
result = re.split(r'([,.]|\s)', text)

print(result)

['Hello', ' ', 'world', ' ', 'this', ' ', 'is', ' ', 'some', ' ', 'sample', ' ', 'text', '.', '']


we need to have spaces, commas and fullstops as tokens as well.

In [311]:
result = [item for item in result if item.strip()]
print(result)

['Hello', 'world', 'this', 'is', 'some', 'sample', 'text', '.']


In [312]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [313]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [314]:
print(len(preprocessed))

4690


## Converting tokens into token ID

In [315]:
all_words = sorted(set(preprocessed))

vocab = {word: i for i, word in enumerate(all_words)}

In [316]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [317]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [318]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

Special tokens:

<|unk|> - unknown token
<|endoftext|> - end of text

In [319]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [320]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int.get(s, self.str_to_int["<|unk|>"]) for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str.get(i, "<|unk|>") for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [321]:
text_str_1 = "hello world this is a test"
text_str_2 = "hello world this is a test. This is a test, too."
text_str = "|<endoftext>|".join((text_str_1, text_str_2))
print(text_str)

tokenizer = SimpleTokenizerV2(vocab)
ids = tokenizer.encode(text_str)
print(ids)

hello world this is a test|<endoftext>|hello world this is a test. This is a test, too.
[1131, 1131, 999, 584, 115, 1131, 1131, 999, 584, 115, 1131, 7, 97, 584, 115, 1131, 5, 1020, 7]


In [322]:
tokenizer.decode(ids)

'<|unk|> <|unk|> this is a <|unk|> <|unk|> this is a <|unk|>. This is a <|unk|>, too.'

## Byte Pair Encoding

- Tokenizers: BPE Tokenizers breaks down unknown words into subwords and characters

In [323]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [324]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


By specifiying the allowed special tokens, we can control the behaviour of the tokenizer 

In [325]:
strings = tokenizer.decode(integers)
strings

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

## Data Sampling with sliding window

In [326]:
with open("../data/the_verdict.txt", "r") as f:
    raw_text = f.read()

encoded_text = tokenizer.encode(raw_text)
print(len(encoded_text))

5145


In [327]:
enc_sample = encoded_text[:50]
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]
print(f"X: {x}")
print(f"Y: {y}")

X: [40, 367, 2885, 1464]
Y: [367, 2885, 1464, 1807]


In [328]:
for i in range(1, context_size+1):
    x = enc_sample[:i]
    y = enc_sample[i]
    print(f"{tokenizer.decode(x)} --> {tokenizer.decode([y])}")

I -->  H
I H --> AD
I HAD -->  always
I HAD always -->  thought


In [329]:
from torch.utils.data import Dataset, DataLoader
import torch
class GPTDatasetV1(Dataset):
    def __init__(self, text, tokenizer, max_length, step):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, step):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [330]:
def create_dataloader_v1(text, batch_size, max_length, step, shuffle, drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(text, tokenizer, max_length, step)
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle, 
        drop_last=drop_last, 
        num_workers=num_workers
    )

    return dataloader

In [331]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=4, step=4, shuffle=False
)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print(f"Inputs:\n{inputs}")
print(f"Targets:\n{targets}")

Inputs:
tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Targets:
tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## Token Embeddings

In [332]:
import torch

vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [333]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, step=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 4, 256])

In [334]:
token_embeddings[0]

tensor([[-0.8697,  0.2042,  0.0188,  ..., -0.4707,  0.5757,  1.8364],
        [ 1.5890,  0.6908,  0.4115,  ..., -0.9212, -1.1576, -0.3751],
        [ 0.2456, -0.1495, -0.0214,  ..., -1.5228,  0.2784, -0.0742],
        [-0.9140,  0.1701, -0.6749,  ..., -0.5430,  0.3704, -0.7477]],
       grad_fn=<SelectBackward0>)

### postional embeddings

In [335]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)


In [336]:
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
pos_embeddings.shape

torch.Size([4, 256])

In [337]:
input_embeddings = token_embeddings + pos_embeddings #possible due to broadcasting
input_embeddings.shape

torch.Size([8, 4, 256])